In [26]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch

# Load the Indonesian Sentence BERT model
model = SentenceTransformer('firqaaa/indo-sentence-bert-base')

In [ ]:
def similarity(word1, word2):
    # Ensure the model and words are on the same device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # Convert words to embeddings
    embeddings1 = model.encode([word1], convert_to_tensor=True, device=device)
    embeddings2 = model.encode([word2], convert_to_tensor=True, device=device)
    
    # Calculate cosine similarity
    sim_score = cosine_similarity(embeddings1.cpu().numpy(), embeddings2.cpu().numpy())[0][0]
    
    return sim_score

In [30]:
# Example usage
word1 = "buku"
word2 = "kitab"
print(similarity(word1, word2))

0.77462053


In [36]:
def load_text_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        # Remove newline characters from each line and strip leading/trailing whitespaces
        lines = [line.strip() for line in lines]
    return lines

# Example usage:
file_path = 'kata.txt'  # Replace 'example.txt' with the path to your text file
vocab_words = load_text_file(file_path)
print(vocab_words[:5])

['a', 'ab', 'aba', 'aba-aba', 'abad']


In [52]:
# List of target words
target_words = ["pendidikan","keluarga", "makanan", "perayaan", "alam"
                ]

In [58]:
import pandas as pd
from tqdm import tqdm

# Encode all vocab words and target words at once
vocab_embeddings = model.encode(vocab_words, convert_to_tensor=True, batch_size=32)
target_embeddings = model.encode(target_words, convert_to_tensor=True, batch_size=32)

# Calculate cosine similarities
# cosine_similarity function can handle batch operations, so no need for a loop
similarities = cosine_similarity(target_embeddings.cpu().numpy(), vocab_embeddings.cpu().numpy())

# Convert similarities to a DataFrame
similarity_df = pd.DataFrame(similarities.T, index=vocab_words, columns=target_words)

similarity_df.head()

,pendidikan,keluarga,makanan,perayaan,alam
a,0.611886,0.316088,0.471980,0.423333,0.588911
ab,0.601733,0.261370,0.381113,0.385298,0.559840
aba,0.575564,0.190402,0.391935,0.333788,0.434451
aba-aba,0.406844,0.129398,0.225260,0.283624,0.285817
abad,0.476435,0.171624,0.265396,0.292270,0.349813


In [60]:
similarity_df.to_csv('game.csv')